Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import catboost as cb
from datetime import datetime
import seaborn as sns

In [2]:
data_train = pd.read_csv("./data/train.csv")
data_test = pd.read_csv("./data/test.csv")
data_webstat = pd.read_csv("./data/t1_webstat.csv")

is_callcenter - был ли совершен звонок в коллцентр по итогам сессии (нужно предсказать)

order_id - id заказа

create_time - время создания заказа

good_id - id товара

price - цена товара

utm_medium - тип трафика

utm_source - источник трафика

sessionkey_id - id сессии пользователя

category_id - id категории товара

parent_id - id родительской категории для категории товара

root_id - id "базовой" категории товара

rating_value - рейтинг товара

rating_count - число оценок товара

description_length - длина описания товара

goods_qty - число товаров в заказе

model_id - ???

is_moderated - ???

pics_qty - ???

model_create_time - ???

sessionkey_id - id сессии пользователя

date_time - timestamp захода на страницу определенного типа

page_type - тип просмотренной страницы

pageview_number - порядковый номер страницы среди всех страниц, отсортированных по времени посещения внутри сессии

pageview_duration_sec - время просмотра страницы

product_in_sale - ???

In [3]:
data_train["create_time_ts"] = None
data_test["create_time_ts"] = None
data_train["model_create_time_ts"] = None
data_test["model_create_time_ts"] = None

from datetime import datetime
# convert string to date
data_train.create_time_ts = data_train.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.create_time_ts = data_test.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

data_train.loc[~data_train.model_create_time.isna(), "model_create_time_ts"] = data_train.loc[~data_train.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.loc[~data_test.model_create_time.isna(), "model_create_time_ts"] = data_test.loc[~data_test.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

In [4]:
del data_train["create_time"]
del data_train["model_create_time"]
del data_test["create_time"]
del data_test["model_create_time"]

In [5]:
data_train.model_create_time_ts = data_train.model_create_time_ts.fillna(data_train.create_time_ts)
data_test.model_create_time_ts = data_test.model_create_time_ts.fillna(data_test.create_time_ts)

In [123]:
print("Рейтинг товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["rating_value"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["rating_value"].mean())

Рейтинг товаров, по которым позвонили:  1.5399876007439555 , по которым не звонили 1.4771690174232548


In [124]:
print("Длина описания товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["description_length"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["description_length"].mean())

Длина описания товаров, по которым позвонили:  572.92983638373 , по которым не звонили 573.338671921299


In [125]:
print("Представителей 0 класса: 67496, 1 класса: 37099")

Представителей 0 класса: 67496, 1 класса: 37099


In [15]:
A = data_train.groupby(['root_id'])['is_callcenter'].sum().sort_values()
B = data_train.groupby(['root_id'])['is_callcenter'].count().sort_values()
C = A.divide(B).sort_values()

order_id = list(data_train["order_id"].unique())
new_column = list()
for order in order_id:
    root = list(data_train[data_train["order_id"] == order]["root_id"].values)[0]
    new_column.append(C[root])
new_column_test = list()
for order in list(data_test["order_id"].unique()):
    root = list(data_test[data_test["order_id"] == order]["root_id"].values)[0]
    new_column_test.append(C[root])
data_train['root_mean_call'] = new_column
data_test['root_mean_call'] = new_column_test

In [27]:
new_column = list()
for order in range(data_train.shape[0]):
    hour = data_train["create_time_ts"][order].time().hour
    new_column.append(hour)
data_train['hour'] = new_column
new_column = list()
for order in range(data_test.shape[0]):
    hour = data_test["create_time_ts"][order].time().hour
    new_column.append(hour)
data_test['hour'] = new_column

In [54]:
data_train["utm_source"].fillna(0, inplace=True)
data_test["utm_source"].fillna(0, inplace=True)

In [66]:
data_train["rating_value"].fillna(0, inplace=True)
data_test["rating_value"].fillna(0, inplace=True)

In [75]:
data_train["rating_count"].fillna(0, inplace=True)
data_test["rating_count"].fillna(0, inplace=True)

In [149]:
new_column = list()
for order in range(data_train.shape[0]):
    day = data_train["create_time_ts"][order].date().weekday()
    new_column.append(day)
data_train['day_of_week'] = new_column
new_column = list()
for order in range(data_test.shape[0]):
    day = data_test["create_time_ts"][order].date().weekday()
    new_column.append(day)
data_test['day_of_week'] = new_column

In [164]:
data_train["life_time"] = (data_train.create_time_ts - data_train.model_create_time_ts).astype('timedelta64[D]')
data_test["life_time"] = (data_test.create_time_ts - data_test.model_create_time_ts).astype('timedelta64[D]')

In [203]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    max_time = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].max()
    new_column.append(max_time)
data_train['max_time_in_page'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    max_time = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].max()
    new_column.append(max_time)
data_test['max_time_in_page'] = new_column

In [ ]:
data_train["max_time_in_page"].fillna(-1, inplace=True)
data_test["max_time_in_page"].fillna(-1, inplace=True)

In [263]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_page'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_page'] = new_column

In [264]:
data_train["count_page"].fillna(0, inplace=True)
data_test["count_page"].fillna(0, inplace=True)

In [229]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_train['sum_time_page'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_test['sum_time_page'] = new_column

In [230]:
data_train["sum_time_page"].fillna(0, inplace=True)
data_test["sum_time_page"].fillna(0, inplace=True)

In [244]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].mean()
    new_column.append(count_page)
data_train['mean_time_page'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].mean()
    new_column.append(count_page)
data_test['mean_time_page'] = new_column

In [245]:
data_train["mean_time_page"].fillna(0, inplace=True)
data_test["mean_time_page"].fillna(0, inplace=True)

In [258]:
data_webstat[data_webstat["sessionkey_id"] == 123777004]

,sessionkey_id,date_time,page_type,pageview_number,pageview_duration_sec,category_id,model_id,good_id,price,product_in_sale
644278,123777004,1975-12-26 09:21:03.393,1,1,NaN,139.0,123517.0,9896348.0,753.0,1.0


In [265]:
features = ["pics_qty", 
            "goods_qty", 
            "description_length", 
            "is_moderated", 
            "model_id", 
            "parent_id", 
            "category_id", 
            "utm_medium", 
            "price", 
            "root_mean_call",
            "hour",
            "utm_source",
            "rating_value",
            "day_of_week",
            "life_time",
            "max_time_in_page",
            "count_page",
            "sum_time_page",
            "mean_time_page"]

# ОБУЧЕНИЕ (TEST)

In [266]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import cv
from catboost import Pool

In [267]:
X_train_val, X_test, y_train_val, y_test = train_test_split(data_train[features], data_train["is_callcenter"], test_size=0.15, random_state=42, stratify=data_train["is_callcenter"])
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42, stratify = y_train_val)

In [268]:
model = CatBoostClassifier(eval_metric = 'AUC', class_weights = [0.35, 0.65], verbose = 100)

In [269]:
model.fit(X_train, y_train,eval_set=(X_val, y_val), use_best_model=True)

Learning rate set to 0.092227
0:	test: 0.7339382	best: 0.7339382 (0)	total: 14.8ms	remaining: 14.8s
100:	test: 0.7869656	best: 0.7869656 (100)	total: 1.11s	remaining: 9.86s
200:	test: 0.7977099	best: 0.7977099 (200)	total: 2.19s	remaining: 8.69s
300:	test: 0.8009489	best: 0.8009489 (299)	total: 3.18s	remaining: 7.39s
400:	test: 0.8026337	best: 0.8026337 (400)	total: 4.17s	remaining: 6.23s
500:	test: 0.8034267	best: 0.8034764 (498)	total: 5.19s	remaining: 5.17s
600:	test: 0.8043867	best: 0.8043867 (600)	total: 6.37s	remaining: 4.23s
700:	test: 0.8042596	best: 0.8046188 (645)	total: 7.41s	remaining: 3.16s
800:	test: 0.8043924	best: 0.8046188 (645)	total: 8.41s	remaining: 2.09s
900:	test: 0.8047239	best: 0.8047290 (899)	total: 9.41s	remaining: 1.03s
999:	test: 0.8047306	best: 0.8047490 (905)	total: 10.4s	remaining: 0us

bestTest = 0.8047490099
bestIteration = 905

Shrink model to first 906 iterations.


In [270]:
predictions = model.predict(X_test[features])

In [271]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predictions)

0.7299947422714718

# ОБУЧЕНИЕ (Kaggle)

In [272]:
X_train, X_val, y_train, y_val = train_test_split(data_train[features], data_train["is_callcenter"], test_size=0.2, random_state=42, stratify=data_train["is_callcenter"])

In [273]:
model = CatBoostClassifier(eval_metric = 'AUC', class_weights = [0.35, 0.65], verbose = 100)
model.fit(X_train, y_train,eval_set=(X_val, y_val), use_best_model=True)

Learning rate set to 0.094578
0:	test: 0.7329923	best: 0.7329923 (0)	total: 18.1ms	remaining: 18s
100:	test: 0.7867498	best: 0.7867498 (100)	total: 1.17s	remaining: 10.4s
200:	test: 0.7968650	best: 0.7968650 (200)	total: 2.22s	remaining: 8.84s
300:	test: 0.8009769	best: 0.8009769 (300)	total: 3.3s	remaining: 7.66s
400:	test: 0.8029251	best: 0.8029251 (400)	total: 4.39s	remaining: 6.56s
500:	test: 0.8044331	best: 0.8044331 (500)	total: 5.48s	remaining: 5.46s
600:	test: 0.8053369	best: 0.8053868 (599)	total: 6.55s	remaining: 4.35s
700:	test: 0.8062876	best: 0.8062984 (699)	total: 7.62s	remaining: 3.25s
800:	test: 0.8067842	best: 0.8068766 (790)	total: 8.85s	remaining: 2.2s
900:	test: 0.8068913	best: 0.8069254 (891)	total: 10s	remaining: 1.1s
999:	test: 0.8070432	best: 0.8070862 (987)	total: 11.2s	remaining: 0us

bestTest = 0.8070861993
bestIteration = 987

Shrink model to first 988 iterations.


In [274]:
predictions = model.predict(data_test[features])

In [275]:
id_column = data_test["order_id"]
result_column = predictions
dataset = pd.DataFrame({'order_id': id_column, 'is_callcenter': list(result_column)}, columns=['order_id', 'is_callcenter'])
dataset.to_csv("sample_submission.csv", index=False)